# Polish Keyboard Layout Optimizer

## Assumptions and goal
Let's dive straight in: our mission is to craft the ultimate keyboard layout for typing in Polish. How do we define "ultimate"? Simple. It's all about minimizing the distance your fingers need to travel while typing. Less travel, more speed, better efficiency.

### What does the best mean?
In the realm of keyboard layouts, "best" is all about efficiency. It's a game of finding a layout where your fingers barely have to move to hit the right keys. Think of it as choreographing a finger ballet—smooth, swift, and efficient.

### Our Target User
Our muse is the classic 10-finger typist, using the traditional home row keys. We're designing for the typist who's all about A, S, D, F, and J, K, L, ; - but with a Polish twist. This means embracing all the unique characters of the Polish language.

### A Note on Variability
Here's where it gets spicy: different datasets lead to different layouts. From formal documents to casual tweets, the variety in language use means our optimal layout might shift a bit. And remember, keyboards themselves are diverse creatures. The spacing and placement of keys can vary, so our ideal layout might not be universally perfect.

## Methodology
It's all about the data. We're diving deep into the CC100-Polish dataset to explore common words and typing patterns, including those unique Polish characters. Feel adventurous? Mix in other datasets for a broader view. Our goal? A keyboard layout that feels like it's reading your mind, with the least amount of finger travel possible.

## Imports

In [1]:
import random
import numpy as np

## Calculate distance between keystrokes

In [13]:
qwerty_layout = {
    'q': (0, 0), 'w': (0, 1), 'e': (0, 2), 'r': (0, 3),
    't': (0, 4), 'y': (0, 5), 'u': (0, 6), 'i': (0, 7), 'o': (0, 8), 'p': (0, 9),
    'a': (1, 0), 's': (1, 1), 'd': (1, 2), 'f': (1, 3),
    'g': (1, 4), 'h': (1, 5), 'j': (1, 6), 'k': (1, 7), 'l': (1, 8), ';': (1, 9),
    'z': (2, 0), 'x': (2, 1), 'c': (2, 2), 'v': (2, 3),
    'b': (2, 4), 'n': (2, 5), 'm': (2, 6), ',': (2, 7), '.': (2, 8), '/': (2, 9)
}

In [3]:
def calculate_keystroke_distance(key1, key2, layout):
    # Define the distances
    standard_distances = {
        'same_col': {(1, 0): 1.032, (0, 1): 1.032, (1, 2): 1.118, (2, 1): 1.118, (2, 0): 2.138, (0, 2): 2.138},  # middle to upper, middle to lower, bottom to top
        'diff_col': {'middle_upper': 1.032, 'middle_lower': 1.118},
    }
    special_cases = {
        ((1, 3), (0, 4)): 1.247, ((1, 5), (0, 6)): 1.247, # F to T, H to U
        ((1, 4), (0, 3)): 1.605, ((1, 6), (0, 5)): 1.605, # G to R, J to Y
        ((2, 4), (1, 3)): 1.803, ((2, 6), (1, 5)): 1.803, # B to F, M to H
        ((2, 4), (0, 3)): 2.661, ((2, 6), (0, 5)): 2.661, # B to R, M to Y
        ((2, 3), (0, 4)): 2.015, ((2, 5), (0, 6)): 2.015, # V to T, N to U
        ((1, 3), (1, 4)): 1.000, ((1, 5), (1, 6)): 1.000  # F to G, J to H
    }
    
    # Get positions
    pos1, pos2 = layout[key1], layout[key2]

    # If the second key is a default position, distance is 0
    if key1 == key2:
        return 0

    # Same column cases
    if pos1[1] == pos2[1]:
        return standard_distances['same_col'][(pos1[0], pos2[0])]
    
    # Adjust for special cases based on columns
    if pos1[1] not in [3, 4] and pos2[1] in [3, 4]:
        pos1 = (1, 3)  # Adjust to F (QWERTY) position
    elif pos1[1] not in [5, 6] and pos2[1] in [5, 6]:
        pos1 = (1, 6)  # Adjust to J (QWERTY) position
        
    # Special cases
    if (pos1, pos2) in special_cases:
        return special_cases[(pos1, pos2)]
    if (pos2, pos1) in special_cases:
        return special_cases[(pos2, pos1)]

    if key2 in 'asdfghjkl; ':
        return 0
    
    # Different column cases
    # Determine middle to top or middle to bottom distance
    if pos2[0] == 0:  # Top row
        return standard_distances['diff_col']['middle_upper']
    elif pos2[0] == 2:  # Bottom row
        return standard_distances['diff_col']['middle_lower']

# Example of using the function
calculate_keystroke_distance('v', 'l', qwerty_layout)

0

## Load Data

In [11]:
def load_and_process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read(2000000)

    # Replace specified characters to non-alt ones
    replacements = {
        '<': ',', 
        '>': '.', 
        ':': ';'
    }
    for old, new in replacements.items():
        data = data.replace(old, new)

    # Convert to lowercase
    data = data.lower()

    # Replace Polish characters
    polish_replacements = {
        'ą': 'a', 'ę': 'e', 'ó': 'o', 
        'ż': 'z', 'ź': 'x', 'ń': 'n', 
        'ć': 'c', 'ś': 's', 'ł': 'l', 
    }
    
    for old, new in polish_replacements.items():
        data = data.replace(old, new)

    # Remove all characters except the specified ones
    allowed_chars = ',.<>;:abcdefghijklmnopqrstuvwxyz'
    data = ''.join([c for c in data if c in allowed_chars])

    # Remove numbers
    data = ''.join([c for c in data if not c.isdigit()])

    return data

# Process the data
file_path = 'pl.txt'  # Adjust the file path if necessary
processed_data = load_and_process_file(file_path)


## Calculate average distance

In [14]:
def calculate_overall_average_distance(data, layout):
    data = data[:len(data) - (len(data) % 100)]
    
    total_distance = 0
    for i in range(len(data) - 1):
        distance = calculate_keystroke_distance(data[i], data[i+1], layout)
        if distance is not None:  # Check if distance is not None
            total_distance += distance

    # Divide the total distance by the number of 100-character segments
    num_segments = len(data) // 100
    return round((total_distance / num_segments), 3) if num_segments > 0 else 0
    
# Calculate the overall average distance of QWERTY layout
overall_average_distance = calculate_overall_average_distance(processed_data, qwerty_layout)
print(f"Overall average distance per 100 characters: {overall_average_distance}")

Overall average distance per 100 characters: 86.338


## Find the best layout

In [7]:
import random

def generate_random_layout(keys):
    return random.sample(keys, len(keys))

def crossover(layout1, layout2):
    cut = random.randint(1, len(layout1) - 2)
    child1_part1 = layout1[:cut]
    child2_part1 = layout2[:cut]

    child1_part2 = [k for k in layout2 if k not in child1_part1]
    child2_part2 = [k for k in layout1 if k not in child2_part1]

    child1 = child1_part1 + child1_part2
    child2 = child2_part1 + child2_part2

    return child1, child2

def mutate(layout):
    index1, index2 = random.sample(range(len(layout)), 2)
    layout[index1], layout[index2] = layout[index2], layout[index1]
    return layout

def layout_list_to_dict(layout_list, base_layout):
    new_layout = {}
    for i, key in enumerate(layout_list):
        new_layout[key] = base_layout[keys[i]]
    return new_layout

def format_print_layout(layout_dict):
    rows = [[], [], []]
    for key, (row, _) in layout_dict.items():
        rows[row].append(key)

    for row in rows:
        print(' '.join(row))

def genetic_algorithm(keys, base_layout, population_size, generations):
    population = [generate_random_layout(keys) for _ in range(population_size)]
    
    for generation in range(generations):
        # Evaluate
        fitness_scores = []
        for layout in population:
            layout_dict = layout_list_to_dict(layout, base_layout)
            score = calculate_overall_average_distance(processed_data, layout_dict)
            fitness_scores.append(score)

        gen_best_index = fitness_scores.index(min(fitness_scores))
        gen_best_layout = population[gen_best_index]
        gen_best_layout_dict = layout_list_to_dict(gen_best_layout, base_layout)
        
        print(f"Generation {generation + 1}: Best layout's average distance per 100 characters = {fitness_scores[gen_best_index]}")
        format_print_layout(gen_best_layout_dict)
        print()

        selected = []
        for _ in range(population_size // 2):
            contenders = random.sample(list(zip(fitness_scores, population)), 3)
            winner = min(contenders, key=lambda x: x[0])[1]
            selected.append(winner)

        offspring = []
        while len(offspring) < population_size // 2:
            parent1, parent2 = random.sample(selected, 2)
            child1, child2 = crossover(parent1, parent2)
            offspring.extend([mutate(child1), mutate(child2)])

        population = selected + offspring

    final_fitness_scores = [calculate_overall_average_distance(processed_data, layout_list_to_dict(layout, base_layout)) for layout in population]
    final_best_index = final_fitness_scores.index(min(final_fitness_scores))
    final_best_layout = population[final_best_index]
    final_best_layout_dict = layout_list_to_dict(final_best_layout, base_layout)

    final_best_score = final_fitness_scores[final_best_index]

    return final_best_layout_dict, final_best_score


# Define the keys and run the genetic algorithm
keys = list(qwerty_layout.keys())
best_layout_dict, best_score = genetic_algorithm(keys, qwerty_layout, population_size=50, generations=35)

# Print the final best layout
print("Best layout:")
format_print_layout(best_layout_dict)
print(f"Average distance per 100 characters: {best_score}")

Generation 1: Best layout's average distance per 100 characters = 51.784
, i s v q . e j l u
r t n d y k ; w o z
a b h c g p m x / f

Generation 2: Best layout's average distance per 100 characters = 49.082
u c z , q t m a ; j
n g i e d / f o p .
b r s w y l v x k h

Generation 3: Best layout's average distance per 100 characters = 47.979
u c z , q t k a ; j
n g i e d / f o p .
b l r h x s w v y m

Generation 4: Best layout's average distance per 100 characters = 47.979
u c z , q t k a ; j
n g i e d / f o p .
b l r h x s w v y m

Generation 5: Best layout's average distance per 100 characters = 47.979
u c z , q t k a ; j
n g i e d / f o p .
b l r h x s w v y m

Generation 6: Best layout's average distance per 100 characters = 45.121
q , i s v ; . e j l
u r t n d y k w o z
f b c g / a p h x m

Generation 7: Best layout's average distance per 100 characters = 45.121
q , i s v ; . e j l
u r t n d y k w o z
f b c g / a p h x m

Generation 8: Best layout's average distance per 100 character